# ANOMALY DETECTION

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import vgg16
from tensorflow.keras.optimizers import Adam, SGD
import scipy  #Used to upsample our image



In [ ]:
import os
import cv2
from PIL import Image
import numpy as np

In [ ]:
image_directory = 'C:\\Users\\user1\\Desktop\\DISSERTATION\\archive\\cell_images\\cell_images'
SIZE = 224
dataset = []  #Many ways to handle data, you can use pandas. Here, we are using a list format.  
label = []  #Placeholders to define add labels. We will add 1 to all parasitized images and 0 to uninfected.



In [ ]:
parasitized_images = os.listdir(image_directory + '\\Parasitized2/')
for i, image_name in enumerate(parasitized_images):    #Remember enumerate method adds a counter and returns the enumerate object
    
    if (image_name.split('.')[1] == 'png'):
        image = cv2.imread(image_directory + '\\Parasitized2/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(1)

In [ ]:
uninfected_images = os.listdir(image_directory + '\\Uninfected2/')
for i, image_name in enumerate(uninfected_images):
    if (image_name.split('.')[1] == 'png'):
        image = cv2.imread(image_directory + '\\Uninfected2/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(0)

In [ ]:
dataset = np.array(dataset)
label = np.array(label)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.20, random_state = 0)

X_train = X_train /255.
X_test = X_test /255.

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
def get_model(input_shape=(224, 224, 3)):
    # Load VGG16 model with ImageNet weights
    vgg = vgg16.VGG16(weights='C:\\Users\\user1\\Desktop\\DISSERTATION\\vgg16_weights_tf_dim_ordering_tf_kernels_notop (1).h5', include_top=False, input_shape=input_shape)

    # Freeze layers up to block4_pool
    for layer in vgg.layers[:-5]:
        layer.trainable = False

    x = vgg.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(2, activation="softmax")(x)

    model = Model(vgg.input, x)
    model.compile(loss="categorical_crossentropy",
                  optimizer=SGD(lr=0.0001, momentum=0.9),
                  metrics=["accuracy"])
    
    return model

model = get_model(input_shape=(224, 224, 3))
print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, batch_size=16, epochs=1, verbose = 1, 
                    validation_data=(X_test,y_test))


In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Check model accuracy on the test data

_, acc = model.evaluate(X_test, y_test)
print("Accuracy = ", (acc * 100.0), "%")

#Test on single image.
n=10  #Select the index of image to be loaded for testing
img = X_test[n]
plt.imshow(img)
input_img = np.expand_dims(img, axis=0) #Expand dims so the input is (num images, x, y, c)
print("The prediction for this image is: ", np.argmax(model.predict(input_img)))
print("The actual label for this image is: ", np.argmax(y_test[n]))

In [ ]:
#Print confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_pred = np.argmax(model.predict(X_test), axis=1)
cm=confusion_matrix(np.argmax(y_test, axis=1), y_pred)  
sns.heatmap(cm, annot=True)

In [ ]:
parasited_image_idx = np.where(y_pred == 1)[0]

#Save all images classified as parasited to a directory (optional, makes sense for large data sets)
#capture it in memory as an array
predicted_as_para=[]
for i in parasited_image_idx:
    par_img = X_test[i]
    #plt.imsave("results_classified_as_para/para_"+str(i)+".png", par_img)
    predicted_as_para.append(par_img)
    
predicted_as_para = np.array(predicted_as_para)    

In [ ]:
from matplotlib.patches import Rectangle  # To add a rectangle overlay to the image
from skimage.feature import peak_local_max  # To detect hotspots in 2D images.

def plot_heatmap(img):
    pred = model.predict(np.expand_dims(img, axis=0))
    pred_class = np.argmax(pred)
    
    # Get weights for all classes from the prediction layer
    last_layer_weights = model.layers[-1].get_weights()[0]  # Prediction layer
    
    # Get weights for the predicted class
    last_layer_weights_for_pred = last_layer_weights[:, pred_class]

    # Get output from the last conv. layer
    last_conv_model = Model(model.input, model.get_layer("block5_conv3").output)
    last_conv_output = last_conv_model.predict(np.expand_dims(img, axis=0))  # Predict on the image
    last_conv_output = np.squeeze(last_conv_output)  # Squeeze the output to remove single-dimensional entries

    # Upsample/resize the last conv. output to the same size as the original image
    h = int(img.shape[0] / last_conv_output.shape[0])
    w = int(img.shape[1] / last_conv_output.shape[1])
    upsampled_last_conv_output = scipy.ndimage.zoom(last_conv_output, (h, w, 1), order=1)

    heat_map = np.dot(upsampled_last_conv_output.reshape((img.shape[0] * img.shape[1], 512)), 
                      last_layer_weights_for_pred).reshape(img.shape[0], img.shape[1])

    # Since we have a lot of dark pixels where the edges may be thought of as 
    # high anomaly, let us drop all heat map values in this region to 0.
    # This is an optional step based on the image. 
    heat_map[img[:, :, 0] == 0] = 0  # All dark pixels outside the object set to 0
    
    # Detect peaks (hot spots) in the heat map. We will set it to detect maximum 5 peaks.
    # with rel threshold of 0.5 (compared to the max peak). 
    peak_coords = peak_local_max(heat_map, num_peaks=5, threshold_rel=0.5, min_distance=10)

    plt.imshow(img.astype('float32').reshape(img.shape[0], img.shape[1], 3))
    plt.imshow(heat_map, cmap='jet', alpha=0.30)
    for i in range(0, peak_coords.shape[0]):
        y = peak_coords[i, 0]
        x = peak_coords[i, 1]
        plt.gca().add_patch(Rectangle((x-25, y-25), 50, 50, linewidth=1, edgecolor='r', facecolor='none'))
        
        


In [ ]:
import random 
im = random.randint(0,predicted_as_para.shape[0]-1)
heat_map =plot_heatmap(predicted_as_para[im])

img = predicted_as_para[im]
plt.imshow(predicted_as_para[im])